## 앞으로 할 것

1. 데이터프레임에서 필요한 행들만 뽑아와서 토빈의 큐와 per 열 이 두가지 열을 만들고 
2. for문으로 모든 기업 데이터프레임 만들기 ..!

In [1]:
import requests # 웹 페이지 소스를 얻기 위한 패키지(기본 내장 패키지이다.)
from bs4 import BeautifulSoup # 웹 페이지 소스를 얻기 위한 패키지, 더 간단히 얻을 수 있다는 장점이 있다고 한다.
from datetime import datetime                                # (!pip install beautifulsoup4 으로 다운받을 수 있다.)
import pandas as pd # 데이터를 처리하기 위한 가장 기본적인 패키지
import time # 사이트를 불러올 때, 작업 지연시간을 지정해주기 위한 패키지이다. (사이트가 늦게 켜지면 에러가 발생하기 때문)
import urllib.request #
from selenium.webdriver import Chrome
import json
import re     
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
import datetime as dt

In [2]:
browser  = Chrome()
browser.maximize_window()

def stock_crawler(code):
    #code = 종목번호
    name = code
    base_url = 'https://finance.naver.com/item/coinfo.nhn?code='+ name + '&target=finsum_more'
    
    browser.get(base_url)
    #frmae구조 안에 필요한 데이터가 있기 때문에 해당 데이터를 수집하기 위해서는 frame구조에 들어가야한다.
    browser.switch_to_frame(browser.find_element_by_id('coinfo_cp'))
    
    #재무제표 "연간" 클릭하기
    browser.find_elements_by_xpath('//*[@class="schtab"][1]/tbody/tr/td[3]')[0].click()

    html0 = browser.page_source
    html1 = BeautifulSoup(html0,'html.parser')
    
    #기업명 뽑기
    title0 = html1.find('head').find('title').text
    print(title0.split('-')[-1])
    
    html22 = html1.find('table',{'class':'gHead01 all-width','summary':'주요재무정보를 제공합니다.'})
    
    #date scrapy
    thead0 = html22.find('thead')
    tr0 = thead0.find_all('tr')[1]
    th0 = tr0.find_all('th')
    
    date = []
    for i in range(len(th0)):
        date.append(''.join(re.findall('[0-9/]',th0[i].text)))
    
    #columns scrapy
    tbody0 = html22.find('tbody')
    tr0 = tbody0.find_all('tr')
    
    col = []
    for i in range(len(tr0)):

        if '\xa0' in tr0[i].find('th').text:
            tx = re.sub('\xa0','',tr0[i].find('th').text)
        else:
            tx = tr0[i].find('th').text

        col.append(tx)
    
    #main text scrapy
    td = []
    for i in range(len(tr0)):
        td0 = tr0[i].find_all('td')
        td1 = []
        for j in range(len(td0)):
            if td0[j].text == '':
                td1.append('0')
            else:
                td1.append(td0[j].text)

        td.append(td1)
    
    td2 = list(map(list,zip(*td)))
    
    return pd.DataFrame(td2,columns = col,index = date)

In [3]:
stock_crawler('005930')

<ipython-input-2-179eb0c8bc39>:11: DeprecationWarning: use driver.switch_to.frame instead
  browser.switch_to_frame(browser.find_element_by_id('coinfo_cp'))


 기업개요(삼성전자)


,매출액,영업이익,영업이익(발표기준),세전계속사업이익,당기순이익,당기순이익(지배),당기순이익(비지배),자산총계,부채총계,자본총계,...,부채비율,자본유보율,EPS(원),PER(배),BPS(원),PBR(배),현금DPS(원),현금배당수익률,현금배당성향(%),발행주식수(보통주)
2016/12,"2,018,667","292,407","292,407","307,137","227,261","224,157","3,104","2,621,743","692,113","1,929,630",...,35.87,"22,004.14","2,735",13.18,"26,636",1.35,570,1.58,17.81,"7,033,966,850"
2017/12,"2,395,754","536,450","536,450","561,960","421,867","413,446","8,422","3,017,521","872,607","2,144,914",...,40.68,"24,536.12","5,421",9.40,"30,427",1.67,850,1.67,14.09,"6,454,924,700"
2018/12,"2,437,714","588,867","588,867","611,600","443,449","438,909","4,540","3,393,572","916,041","2,477,532",...,36.97,"27,531.92","6,024",6.42,"35,342",1.09,"1,416",3.66,21.92,"5,969,782,550"
2019/12,"2,304,009","277,685","277,685","304,322","217,389","215,051","2,338","3,525,645","896,841","2,628,804",...,34.12,"28,856.02","3,166",17.63,"37,528",1.49,"1,416",2.54,44.73,"5,969,782,550"
2020/12,"2,368,070","359,939","359,939","363,451","264,078","260,908","3,170","3,782,357","1,022,877","2,759,480",...,37.07,"30,692.79","3,841",21.09,"39,406",2.06,"2,994",3.70,77.95,"5,969,782,550"
2021/12,"2,633,141","474,219",0,"489,210","355,639","351,446",0,"4,036,271","1,083,984","2,952,287",...,36.72,0,"5,174",16.39,"42,230",2.01,"1,727",2.04,29.34,0
2022/12,"2,902,887","613,994",0,"627,960","458,395","453,319",0,"4,394,170","1,134,716","3,274,455",...,34.65,0,"6,674",12.71,"46,786",1.81,"2,010",2.37,26.47,0
2023/12,"2,967,011","613,603",0,"638,082","467,498","461,608",0,"4,679,007","1,164,989","3,514,016",...,33.15,0,"6,796",12.48,"49,987",1.70,"2,352",2.77,30.42,0
